In [1]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    UnstructuredHTMLLoader,
    JSONLoader,
    UnstructuredMarkdownLoader,
)
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [8]:
def accept_document_return_chunks(document: str):
    """
    _description_
    This function accepts a document, creates chunks, embed the chunks and write to a vector store

    Parameters
    ----------
    document : str
        _description_
        takes the directory of the document to be processed

    Returns
    -------
    _type_
        _description_
        Chunks of text from the document

    Raises
    ------
    ValueError
        _description_
    """

    if document.split(".")[-1] == "pdf":
        loader = PyPDFLoader(document)
    elif document.split(".")[-1] == "txt":
        loader = TextLoader(document)
    elif document.split(".")[-1] == "json":
        loader = JSONLoader(document)
    elif document.split(".")[-1] == "html":
        loader = UnstructuredHTMLLoader(document)
    elif document.split(".")[-1] == "md":
        loader = UnstructuredMarkdownLoader(document)
    else:
        raise ValueError("File type not supported")

    pages = loader.load_and_split()

    unify_content = ""
    for page in pages:
        unify_content += "\n" + page.page_content

    # embedder = SentenceTransformerEmbeddings(
    #     model_name="Dr-BERT/DrBERT-7GB",
    # )

    embedder = SentenceTransformerEmbeddings(
        model_name="dangvantuan/sentence-camembert-large",
    )

    semantic_splitter = SemanticChunker(embedder)

    docs = semantic_splitter.create_documents([unify_content])

    # text_splitter = RecursiveCharacterTextSplitter(
    #     # Set a really small chunk size, just to show.
    #     chunk_size=500,
    #     chunk_overlap=20,
    #     length_function=len,
    #     is_separator_regex=False,
    # )

    # docs2 = text_splitter.create_documents([unify_content])

    patient_id = document.split("/")[-1].split(".")[0]

    # docs = docs + ["patient_id": patient_id]

    print(patient_id, docs)

    db = Chroma.from_documents(docs, embedder, persist_directory="./inputs")

    return db  # , docs2


data = accept_document_return_chunks("lettre_6.txt")

No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with MEAN pooling.


lettre_6 [Document(page_content='\nGROSSESSE EN COURS: 3 ° grossesse, suivie par Dr CLAUDOT Alain\n\nDDR : 11/02/2016, ovulation: 4/03/2016,\nterme calculé: 17/11/2016, terme corrigé: 25/11/2016 \nTabac: non, Alcool: non, Toxicomanie: non, Antécédent de césarienne: non\nObésité: oui si BMI >= 30; valeur du BMI: 24.4 \nDépistage diabète gestationnel : 85; OGTT normal\nEcouvillon vaginal : 31/5: strepto +\tkiné prénatale :non\nrisque biologique trisomie 21: 1/1007, risque intégré de Tri 21: 1/4060 risque biologique trisomie 18: <1/10000 Risque NTD: <1/10000\nPhénotype Rhésus Foetal par PCR: Rh D positif, Chrom Y positif  \n\nExamen bassin : \n\n\nrésumé de la grossesse:\nPoids de départ: 61, Taille: 158, soit un BMI à 24.4,\n{SEROLOGIE POSITIVE pour {HIV|Hépatite B|Hépatite C|@}\n}Donné Rhogam prophylactique 28 sem, Boostrix injecté vers 24-32 sem, \n\nconduite à tenir :\n \n{Assurance hospitalisation: non \n}{Suivi Tabacologue: non\n}{Reconnaissance anticipée (état civil: Mariée): non f

InvalidDimensionException: Embedding dimension 1024 does not match collection dimensionality 384

In [4]:
print(data.get())

{'ids': ['8a03dd1e-e091-11ee-931d-48f17fff437b', 'db9dac64-e090-11ee-931d-48f17fff437b'], 'embeddings': None, 'metadatas': [None, None], 'documents': ['\nGROSSESSE EN COURS: 3 ° grossesse, suivie par Dr CLAUDOT Alain\n\nDDR : 11/02/2016, ovulation: 4/03/2016,\nterme calculé: 17/11/2016, terme corrigé: 25/11/2016 \nTabac: non, Alcool: non, Toxicomanie: non, Antécédent de césarienne: non\nObésité: oui si BMI >= 30; valeur du BMI: 24.4 \nDépistage diabète gestationnel : 85; OGTT normal\nEcouvillon vaginal : 31/5: strepto +\tkiné prénatale :non\nrisque biologique trisomie 21: 1/1007, risque intégré de Tri 21: 1/4060 risque biologique trisomie 18: <1/10000 Risque NTD: <1/10000\nPhénotype Rhésus Foetal par PCR: Rh D positif, Chrom Y positif  \n\nExamen bassin : \n\n\nrésumé de la grossesse:\nPoids de départ: 61, Taille: 158, soit un BMI à 24.4,\n{SEROLOGIE POSITIVE pour {HIV|Hépatite B|Hépatite C|@}\n}Donné Rhogam prophylactique 28 sem, Boostrix injecté vers 24-32 sem, \n\nconduite à tenir :

In [ ]:
/output/load.txt